In [3]:
import pandas as pd
import geopandas as gpd

In [245]:
from bokeh.plotting import figure, show
from bokeh.models import (
    LinearAxis,
    Range1d,
    Tabs,
    TabPanel,
    ColumnDataSource,
    HoverTool,
)


def get_multi_tab_line_plot(
    df1, df2, loop_list, category_column, conflict_measure, ntl_measure
):
    tabs = []

    for ts in loop_list:
        # # Create a new Bokeh figure
        p = figure(
            title=f"Comparing Nighttime Light Trends and {conflict_measure.capitalize()}",
            x_axis_label="Month",
            width=800,
            height=400,
            x_axis_type="datetime",
        )

        p.y_range.start = df2[df2[category_column] == ts][conflict_measure].min()
        p.y_range.end = df2[df2[category_column] == ts][conflict_measure].max()

        p.extra_y_ranges = {
            "y2": Range1d(
                start=df1[df1[category_column] == ts][ntl_measure].min(),
                end=df1[df1[category_column] == ts][ntl_measure].max(),
            )
        }
        # Add the second line plot
        ntl_source = ColumnDataSource(df1[df1[category_column] == ts])
        ntl_line = p.vbar(
            "date",
            top=ntl_measure,
            source=ntl_source,
            legend_label=ntl_measure,
            width=2000 * 2000 * 750,
            color="blue",
            alpha=0.7,
            y_range_name="y2",
        )

        # Add the first line plot
        conflict_source = ColumnDataSource(df2[df2[category_column] == ts])
        conflict_line = p.line(
            "date",
            conflict_measure,
            source=conflict_source,
            legend_label=f"{conflict_measure.capitalize()}",
            line_width=2,
            color="red",
            alpha=0.7,
        )

        # Customize legend
        p.legend.location = "top_left"
        p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

        p.add_layout(LinearAxis(y_range_name="y2", axis_label="Luminosity"), "right")

        # Add hover tool for conflict data
        hover_conflict = HoverTool(
            renderers=[conflict_line],  # Reference the renderer directly
            tooltips=[
                ("Date", "@date{%F}"),
                (f"{conflict_measure.capitalize()}", f"@{conflict_measure}"),
            ],
            formatters={
                "@date": "datetime",  # Format the date
            },
            mode="vline",
        )

        # Add hover tool for NTL data
        hover_ntl = HoverTool(
            renderers=[ntl_line],  # Reference the renderer directly
            tooltips=[
                ("Date", "@date{%F}"),
                (ntl_measure, f"@{ntl_measure}"),
            ],
            formatters={
                "@date": "datetime",  # Format the date
            },
            mode="vline",
        )

        p.add_tools(hover_conflict, hover_ntl)

        tab = TabPanel(child=p, title=ts)
        tabs.append(tab)

    return tabs

In [276]:
def get_multi_tab_bar_plot(df1, loop_list, category_column, ntl_measure):
    tabs = []

    for ts in loop_list:
        # # Create a new Bokeh figure
        p = figure(
            title="Nighttime Light Trends",
            x_axis_label="Month",
            width=800,
            height=400,
            x_axis_type="datetime",
        )

        ntl_source = ColumnDataSource(df1[df1[category_column] == ts])
        p.y_range.start = df1[df1[category_column] == ts][ntl_measure].min()
        p.y_range.end = df1[df1[category_column] == ts][ntl_measure].max()
        # Add the second line plot

        ntl_line = p.vbar(
            x="date",
            top=ntl_measure,
            source=ntl_source,
            legend_label=ntl_measure,
            width=2000 * 2000 * 750,
            color="blue",
            alpha=0.7,
        )

        # Customize legend
        p.legend.location = "top_left"
        p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

        # Add hover tool for NTL data
        hover_ntl = HoverTool(
            renderers=[ntl_line],  # Reference the renderer directly
            tooltips=[
                ("Date", "@date{%F}"),
                (ntl_measure, f"@{ntl_measure}"),
            ],
            formatters={
                "@date": "datetime",  # Format the date
            },
            mode="vline",
        )

        p.add_tools(hover_ntl, hover_ntl)

        tab = TabPanel(child=p, title=ts)
        tabs.append(tab)

    return tabs

# Conflict and Nighttime Lights

This notebook analyses how Nighttime Lights changed as the conflict in the country changed. Conflict is measured using ACLED data and a conflict index is calculated as geometric mean between Number of fatalities and Number of events. This conflict index is then compred to Nighttime lights (measured as lumiinosity) wihout gas flaring. The Nighttime Lights data is retrieved from [BlackMarble](https://datapartnership.org/myanmar-economic-monitor/notebooks/nighttime-lights/analysis-2023/README.html). 

## Insights

The following questions quide the data analysis - 

- **How is NTL different in EAO controlled areas and other regions?**
For this, we used the control map of Myanmar as of March 2024 released by the [Special Advisory Council](https://specialadvisorycouncil.org/wp-content/uploads/2024/05/SAC-M-Effective-Control-in-Myanmar-2024-Update-ENGLISH.pdf). The contestation and control is divided into 8 regions starting from 'stable junta control' to 'complete resistance control'. We expect to see that the regions that have moved into 'complete resistance control' have also experienced high increase in conflict. 

- **Is there a direct correlation between Conflict Index and Nighttime Lights?**
We did not find a direct correlation between the two variables at a national or township level.

- **How did NTL change in regions that saw high increase in conflict compared to 6 months prior?**
There is no direct correlation between the two variables found yet. 



### Nighttime Lights in EAO Controlled Areas Vs Junta Controlled Areas

In [271]:
ntl_adm3_monthly = pd.read_csv("../../data/ntl/ntl_mmr_adm3_monthly.csv")
conflict_monthly = pd.read_stata(
    "../../data/conflict/ACLED_01_Jan_2021_21_Oct_2024_township_monthly_spatial.dta"
)
conflict_6_monthly = pd.read_stata(
    "../../data/conflict/ACLED_Apr_2022_Sep_2024_township_spatial_period_change.dta"
)
mmr_adm3 = gpd.read_file("../../data/boundaries/mmr_polbnda_adm3_250k_mimu.shp")

In [272]:
ntl_adm3_monthly = mmr_adm3[["ST", "DT", "TS", "geometry"]].merge(
    ntl_adm3_monthly, on=["ST", "DT", "TS"]
)
ntl_adm3_monthly["date"] = pd.to_datetime(ntl_adm3_monthly["date"])
conflict_monthly.rename(columns={"month": "date"}, inplace=True)

In [273]:
control_categories = pd.read_csv("../../data/conflict/control_category_map.csv")
control_categories.rename(columns={"Township": "TS"}, inplace=True)
eao_controlled = list(
    control_categories[control_categories["code_no"] == 8.0]["TS"].unique()
)
junta_controlled = list(
    control_categories[control_categories["code_no"] == 1.0]["TS"].unique()
)

In [274]:
ntl_adm3_monthly = ntl_adm3_monthly.merge(control_categories, on="TS")
ntl_adm3_monthly["code_no"] = ntl_adm3_monthly["code_no"].astype(str)

In [284]:
conflict_monthly = conflict_monthly.merge(control_categories, on="TS")
conflict_monthly["code_no"] = conflict_monthly["code_no"].astype(str)

In [287]:
control_code_dict = {
    "1.0": "Stable junta control",
    "2.0": "Junta dependent on local proxy militias for control",
    "3.0": "Junta forces under regular attack from resistance forces; administration functions remain weak",
    "4.0": "Resistance controls growing territory but still cannot consolidate fuller control",
    "5.0": "Limited junta movement, dependent on ceasefires",
    "6.0": "Junta control receding; resistance defending increasing territories & asserting local administration",
    "7.0": "Strong resistance control & local administration—90%+ of township",
    "8.0": "Full resistance control & local administration—whole township",
}

- 1:Stable junta control
- 2:Junta dependent on local proxy militias for control
- 3:Junta forces under regular attack from resistance forces; administration functions remain weak
- 4:Resistance controls growing territory but still cannot consolidate fuller control
- 5:Limited junta movement, dependent on ceasefires
- 6:Junta control receding; resistance defending increasing territories & asserting local administration
- 7:Strong resistance control & local administration—90%+ of township
- 8:Full resistance control & local administration—whole township

In [288]:
# Import necessary libraries
output_notebook()

df1 = (
    ntl_adm3_monthly[["TS", "code_no", "date", "ntl_sum", "ntl_nogf_5km_sum"]]
    .groupby(["date", "code_no"])[["ntl_sum", "ntl_nogf_5km_sum"]]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-1"]
ntl_measure = "ntl_nogf_5km_sum"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=list(control_code_dict.keys()),
    category_column="code_no",
    ntl_measure=ntl_measure,
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

**There is a huge peak in the currently EAO controlled regions back in April 2023 where there is massive peak in nighttime lights. The same peak is not visible in the junta controlled regions.** There is a lot more consistent light in regions where junta forces are under regular attack (category 3).

#### Nighttime Lights in EAO Controlled Townships

In [280]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = (
    ntl_adm3_monthly[["TS", "code_no", "date", "ntl_sum", "ntl_nogf_5km_sum"]]
    .groupby(["date", "TS"])[["ntl_sum", "ntl_nogf_5km_sum"]]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-1"]
ntl_measure = "ntl_nogf_5km_sum"


tabs = get_multi_tab_bar_plot(
    df1, loop_list=eao_controlled, category_column="TS", ntl_measure=ntl_measure
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

#### Nighttime Lights in Junta Controlled Townships

In [281]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = (
    ntl_adm3_monthly[["TS", "code_no", "date", "ntl_sum", "ntl_nogf_5km_sum"]]
    .groupby(["date", "TS"])[["ntl_sum", "ntl_nogf_5km_sum"]]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-1"]
ntl_measure = "ntl_nogf_5km_sum"


tabs = get_multi_tab_bar_plot(
    df1, loop_list=junta_controlled, category_column="TS", ntl_measure=ntl_measure
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

### Comparing Trends in Conflict with Nighttime Lights


In [278]:
very_severe_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "Very severe conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)
severe_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "Severe conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)
no_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "No conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)

#### Conflict and Nighttime Lights in EAO Controlled Areas Vs Junta Controlled Areas

In [290]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = (
    ntl_adm3_monthly[["TS", "code_no", "date", "ntl_sum", "ntl_nogf_5km_sum"]]
    .groupby(["date", "code_no"])[["ntl_sum", "ntl_nogf_5km_sum"]]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-1"]
ntl_measure = "ntl_nogf_5km_sum"

df2 = (
    conflict_monthly.groupby(["date", "code_no"], observed=False)[
        ["conflict_index", "fatalities", "events"]
    ]
    .mean()
    .reset_index()
)
df2.dropna(subset="conflict_index", inplace=True)
conflict_measure = "conflict_index"

tabs = get_multi_tab_line_plot(
    df1,
    df2,
    loop_list=list(control_code_dict.keys()),
    category_column="code_no",
    conflict_measure=conflict_measure,
    ntl_measure=ntl_measure,
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [283]:
# # Import necessary libraries
# from bokeh.io import output_notebook
# output_notebook()


# # If you are running this in a Jupyter notebook, uncomment the next line
# # output_notebook()

# df1 = ntl_adm3_monthly.groupby(['date'])[['ntl_sum', 'ntl_gf_5km_sum',
#        'ntl_nogf_5km_sum', 'ntl_gf_10km_sum', 'ntl_nogf_10km_sum']].mean().reset_index()
# df1=df1[df1['date']>'2021-01-1']
# df2 = conflict_monthly.groupby(['date'])[['conflict_index', 'fatalities', 'events']].mean().reset_index()
# ntl_measure = 'ntl_nogf_5km_sum'

# tabs = []

# for id, conflict_measure in enumerate(['conflict_index', 'events', 'fatalities']):

#        # Create a new Bokeh figure
#        p = figure(title=f"Nighttime Light Trends and {conflict_measure.capitalize()} at a National Level", x_axis_label='Month', width=800, height=400, x_axis_type='datetime')

#        p.y_range.start = 0
#        p.y_range.end = df2[conflict_measure].max()

#        # Add the first line plot
#        p.line(df2['date'], df2[conflict_measure], legend_label=f"{conflict_measure.capitalize()}", line_width=2, color="red", alpha=0.7)

#        p.extra_y_ranges = {"y2": Range1d(start=0, end=df1[ntl_measure].max())}
#        # Add the second line plot
#        p.vbar(df1['date'], top=df1[ntl_measure], legend_label=ntl_measure, width=2000*2000*750, color="blue", alpha=0.7, y_range_name='y2')

#        # Customize legend
#        p.legend.location = "top_left"
#        p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

#        p.add_layout(LinearAxis(y_range_name="y2", axis_label='Luminosity'), 'right')

#        tab = TabPanel(child=p, title=conflict_measure.capitalize())
#        tabs.append(tab)

# # Create the Tabs layout
# tabs_layout = Tabs(tabs=tabs)

# # Show the tabs
# show(tabs_layout)

**There is no obvious correlation between NTL and Conflict Index, Events or Fatalities at a national level**. In fact, in some periods it has a positive correlation where the NTL goes up along with conflict. To inspect this further, we broke it down at a regional level. The entire country saw an increase in NTL in February 2023. 

### Conflict and NTL in Very severely and severely conflicted regions

In [282]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = (
    ntl_adm3_monthly.groupby(["date", "TS"])[
        [
            "ntl_sum",
            "ntl_gf_5km_sum",
            "ntl_nogf_5km_sum",
            "ntl_gf_10km_sum",
            "ntl_nogf_10km_sum",
        ]
    ]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-1"]
df2 = (
    conflict_monthly.groupby(["date", "TS", "conflict_category"], observed=False)[
        ["conflict_index", "fatalities", "events"]
    ]
    .mean()
    .reset_index()
)
df2.dropna(subset="conflict_index", inplace=True)
conflict_measure = "conflict_index"
ntl_measure = "ntl_nogf_5km_sum"

tabs = get_multi_tab_line_plot(
    df1,
    df2,
    loop_list=very_severe_conflict,
    category_column="TS",
    conflict_measure=conflict_measure,
    ntl_measure=ntl_measure,
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

**In Maungdaw, there is a corresponding increase of NTL with conflict index in August 2022 and in April 2024. Similarly, there is a decrease in NTL with conflict in October 2022.** The opposite movement is seen in May 2022 when the NTL goes up and the conflict came down. 

**In Lashio, October 2023 saw a corresponding increase in NTL and Conflict.** However, early in 2024 the same level of NTL also has a lower level of conflict. 

No convincing trends can be identified to understand the relationship between these variables yet. A simple correlation plot between conflict index and NTL for severely conflicted locations and very severely conflicted location also reveals that no trend can be seen. 

In [29]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs
from bokeh.plotting import show
from scipy import stats  # For linear regression

# Data preparation
df1 = (
    ntl_adm3_monthly.groupby(["date", "TS"])[
        [
            "ntl_sum",
            "ntl_gf_5km_sum",
            "ntl_nogf_5km_sum",
            "ntl_gf_10km_sum",
            "ntl_nogf_10km_sum",
        ]
    ]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-01"]

df2 = (
    conflict_monthly.groupby(["date", "TS", "conflict_category"], observed=False)[
        ["conflict_index", "fatalities", "events"]
    ]
    .mean()
    .reset_index()
)
df2.dropna(subset="conflict_index", inplace=True)

conflict_measure = "conflict_index"
ntl_measure = "ntl_nogf_5km_sum"

tabs = []

# Loop through severe conflicts and create a scatter plot with a best fit line for each
for ts in very_severe_conflict:
    # Merge df1 and df2 on 'date' and 'TS' to align conflict and NTL data for the scatter plot
    merged_df = pd.merge(df1[df1["TS"] == ts], df2[df2["TS"] == ts], on=["date", "TS"])

    # Create a new Bokeh figure
    p = figure(
        title=f"Conflict vs Nighttime Light (TS={ts})",
        x_axis_label=f"{conflict_measure.capitalize()}",
        y_axis_label=f"{ntl_measure}",
        width=800,
        height=400,
    )

    # Scatter plot with conflict on x-axis and NTL on y-axis using `scatter()`
    scatter_source = ColumnDataSource(merged_df)
    scatter_plot = p.scatter(
        conflict_measure,
        ntl_measure,
        source=scatter_source,
        size=8,
        color="blue",
        alpha=0.7,
    )

    # Add hover tool to show both conflict and NTL values
    hover_scatter = HoverTool(
        renderers=[scatter_plot],
        tooltips=[
            ("Date", "@date{%F}"),
            (f"{conflict_measure.capitalize()}", f"@{conflict_measure}"),
            (ntl_measure, f"@{ntl_measure}"),
        ],
        formatters={"@date": "datetime"},  # Format the date
        mode="mouse",
    )

    p.add_tools(hover_scatter)

    # Calculate the best fit line using scipy's linear regression
    x = merged_df[conflict_measure].values
    y = merged_df[ntl_measure].values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Best fit line points
    best_fit_y = intercept + slope * x

    # Plot the best fit line
    p.line(x, best_fit_y, line_width=2, color="red", legend_label="Best Fit Line")

    # Create a tab for each conflict
    tab = TabPanel(child=p, title=str(ts))
    tabs.append(tab)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

Loading BokehJS ...

In [33]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs, TabPanel, HoverTool, ColumnDataSource
from bokeh.plotting import figure, show
from scipy import stats  # For linear regression

# Data preparation
df1 = (
    ntl_adm3_monthly.groupby(["date", "TS"])[
        [
            "ntl_sum",
            "ntl_gf_5km_sum",
            "ntl_nogf_5km_sum",
            "ntl_gf_10km_sum",
            "ntl_nogf_10km_sum",
        ]
    ]
    .mean()
    .reset_index()
)
df1 = df1[df1["date"] > "2021-01-01"]

df2 = (
    conflict_monthly.groupby(["date", "TS", "conflict_category"], observed=False)[
        ["conflict_index", "fatalities", "events"]
    ]
    .mean()
    .reset_index()
)
df2.dropna(subset="conflict_index", inplace=True)

conflict_measure = "conflict_index"
ntl_measure = "ntl_nogf_5km_sum"

tabs = []

# Loop through severe conflicts and create a scatter plot with a best fit line for each
for ts in severe_conflict:
    # Merge df1 and df2 on 'date' and 'TS' to align conflict and NTL data for the scatter plot
    merged_df = pd.merge(df1[df1["TS"] == ts], df2[df2["TS"] == ts], on=["date", "TS"])

    # Create a new Bokeh figure
    p = figure(
        title=f"{conflict_measure.capitalize()} vs Nighttime Light (TS={ts})",
        x_axis_label=f"{conflict_measure.capitalize()}",
        y_axis_label=f"{ntl_measure}",
        width=800,
        height=400,
    )

    # Scatter plot with conflict on x-axis and NTL on y-axis using `scatter()`
    scatter_source = ColumnDataSource(merged_df)
    scatter_plot = p.scatter(
        conflict_measure,
        ntl_measure,
        source=scatter_source,
        size=8,
        color="blue",
        alpha=0.7,
    )

    # Add hover tool to show both conflict and NTL values
    hover_scatter = HoverTool(
        renderers=[scatter_plot],
        tooltips=[
            ("Date", "@date{%F}"),
            (f"{conflict_measure.capitalize()}", f"@{conflict_measure}"),
            (ntl_measure, f"@{ntl_measure}"),
        ],
        formatters={"@date": "datetime"},  # Format the date
        mode="mouse",
    )

    p.add_tools(hover_scatter)

    # Calculate the best fit line using scipy's linear regression
    x = merged_df[conflict_measure].values
    y = merged_df[ntl_measure].values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Best fit line points
    best_fit_y = intercept + slope * x

    # Plot the best fit line
    p.line(x, best_fit_y, line_width=2, color="red", legend_label="Best Fit Line")

    # Create a tab for each conflict
    tab = TabPanel(child=p, title=str(ts))
    tabs.append(tab)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

Loading BokehJS ...

### Comparing difference in conflict with difference in NTL for a 6-month-period

In [62]:
ntl_adm3_6_monthly = (
    ntl_adm3_monthly.groupby(["TS", "date"])[
        [
            "ntl_sum",
            "ntl_gf_5km_sum",
            "ntl_nogf_5km_sum",
            "ntl_gf_10km_sum",
            "ntl_nogf_10km_sum",
        ]
    ]
    .mean()
    .reset_index()
)

conflict_adm3_6_monthly = (
    conflict_monthly.groupby(["TS", "date"])[["conflict_index", "events", "fatalities"]]
    .mean()
    .reset_index()
)

In [39]:
def assign_6month_group(date):
    # Find the month of the date
    month = date.month
    # Calculate the "group" by checking whether the date falls in Apr-Sep or Oct-Mar periods
    if month >= 4 and month <= 9:
        # Group starting from April
        return f"{date.year}-Apr-Sep"
    else:
        # Group starting from October
        if month >= 10:
            return f"{date.year}-Oct-Mar"
        else:
            return f"{date.year-1}-Oct-Mar"  # For Jan-Mar, it's part of the previous year's Oct-Mar

In [63]:
ntl_adm3_6_monthly["6-month-group"] = ntl_adm3_6_monthly["date"].apply(
    lambda x: assign_6month_group(x)
)
ntl_adm3_6_monthly = (
    ntl_adm3_6_monthly.groupby(["TS", "6-month-group"])[
        [
            "ntl_sum",
            "ntl_gf_5km_sum",
            "ntl_nogf_5km_sum",
            "ntl_gf_10km_sum",
            "ntl_nogf_10km_sum",
        ]
    ]
    .mean()
    .reset_index()
)

conflict_adm3_6_monthly["6-month-group"] = conflict_adm3_6_monthly["date"].apply(
    lambda x: assign_6month_group(x)
)
conflict_adm3_6_monthly = (
    conflict_adm3_6_monthly.groupby(["TS", "6-month-group"])[
        ["conflict_index", "events", "fatalities"]
    ]
    .mean()
    .reset_index()
)

In [182]:
df1 = (
    ntl_adm3_monthly.groupby(["TS", "date"])[
        [
            "ntl_sum",
            "ntl_gf_5km_sum",
            "ntl_nogf_5km_sum",
            "ntl_gf_10km_sum",
            "ntl_nogf_10km_sum",
        ]
    ]
    .mean()
    .reset_index()
)
df1 = df1.sort_values(by=["TS", "date"])
df1["ntl_nogf_5km_diff_pm"] = df1.groupby("TS")["ntl_nogf_5km_sum"].diff()
df1["ntl_diff_pm"] = df1.groupby("TS")["ntl_sum"].diff()
df1["ntl_gf_5km_pm"] = df1.groupby("TS")["ntl_gf_5km_sum"].diff()

In [183]:
df2 = (
    conflict_monthly.groupby(["TS", "date"])[["events", "fatalities", "conflict_index"]]
    .mean()
    .reset_index()
)
df2 = df2.sort_values(by=["TS", "date"])
df2["conflict_index_diff_pm"] = df2.groupby("TS")["conflict_index"].diff()
df2["events_diff_pm"] = df2.groupby("TS")["events"].diff()
df2["fatalities_pm"] = df2.groupby("TS")["fatalities"].diff()

In [184]:
df = ntl_adm3_6_monthly.merge(conflict_adm3_6_monthly, on=["TS", "6-month-group"])
df["ntl_6_month_diff"] = df.groupby("TS")["ntl_nogf_5km_sum"].diff()
df["conflict_6_month_diff"] = df.groupby("TS")["conflict_index"].diff()

In [185]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs, TabPanel, HoverTool, ColumnDataSource


df1 = df1[df1["date"] > "2021-01-1"]
conflict_measure = "conflict_index_diff_pm"
ntl_measure = "ntl_nogf_5km_diff_pm"

tabs = get_multi_tab_line_plot(
    df1, df2, very_severe_conflict, "TS", conflict_measure, ntl_measure
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [186]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs, TabPanel, HoverTool, ColumnDataSource


df1 = df1[df1["date"] > "2021-01-1"]
conflict_measure = "conflict_index_diff_pm"
ntl_measure = "ntl_nogf_5km_diff_pm"

tabs = get_multi_tab_line_plot(
    df1, df2, no_conflict[0:10], "TS", conflict_measure, ntl_measure
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [118]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs, TabPanel, HoverTool, ColumnDataSource


conflict_measure = "conflict_6_month_diff"
ntl_measure = "ntl_6_month_diff"

tabs = []

for ts in very_severe_conflict:
    # # Create a new Bokeh figure
    p = figure(
        title=f"Comparing Nighttime Light Trends and {conflict_measure.capitalize()}",
        x_axis_label="6-Month-Period",
        width=800,
        height=400,
        x_range=sorted(df["6-month-group"].unique()),
    )

    p.y_range.start = df[df["TS"] == ts][conflict_measure].min()
    p.y_range.end = df[df["TS"] == ts][conflict_measure].max()

    # Add the first line plot
    conflict_source = ColumnDataSource(df[df["TS"] == ts])
    conflict_line = p.line(
        "6-month-group",
        conflict_measure,
        source=conflict_source,
        legend_label=f"{conflict_measure.capitalize()}",
        line_width=2,
        color="red",
        alpha=0.7,
    )

    p.extra_y_ranges = {
        "y2": Range1d(
            start=df[df["TS"] == ts][ntl_measure].min(),
            end=df[df["TS"] == ts][ntl_measure].max(),
        )
    }
    # Add the second line plot
    ntl_source = ColumnDataSource(df[df["TS"] == ts])
    ntl_line = p.line(
        "6-month-group",
        ntl_measure,
        source=ntl_source,
        legend_label=ntl_measure,
        line_width=2,
        color="blue",
        alpha=0.7,
        y_range_name="y2",
    )

    # Customize legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

    p.add_layout(LinearAxis(y_range_name="y2", axis_label="Luminosity"), "right")

    # Add hover tool for conflict data
    hover_conflict = HoverTool(
        renderers=[conflict_line],  # Reference the renderer directly
        tooltips=[
            ("Date", "@6-month-group{%F}"),
            (f"{conflict_measure.capitalize()}", f"@{conflict_measure}"),
        ],
        formatters={
            "@6-month-group": "datetime",  # Format the date
        },
        mode="vline",
    )

    # Add hover tool for NTL data
    hover_ntl = HoverTool(
        renderers=[ntl_line],  # Reference the renderer directly
        tooltips=[
            ("Date", "@6-month-group{%F}"),
            (ntl_measure, f"@{ntl_measure}"),
        ],
        formatters={
            "@6-month-group": "datetime",  # Format the date
        },
        mode="vline",
    )

    p.add_tools(hover_conflict, hover_ntl)

    tab = TabPanel(child=p, title=ts)
    tabs.append(tab)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [121]:
# Import necessary libraries
output_notebook()
from bokeh.models import (
    LinearAxis,
    Range1d,
    Tabs,
    TabPanel,
    HoverTool,
    ColumnDataSource,
)


conflict_measure = "conflict_6_month_diff"
ntl_measure = "ntl_6_month_diff"

tabs = []

for ts in no_conflict[0:10]:
    # # Create a new Bokeh figure
    p = figure(
        title=f"Comparing Nighttime Light Trends and {conflict_measure.capitalize()}",
        x_axis_label="6-Month-Period",
        width=800,
        height=400,
        x_range=sorted(df["6-month-group"].unique()),
    )

    p.y_range.start = df[df["TS"] == ts][conflict_measure].min()
    p.y_range.end = df[df["TS"] == ts][conflict_measure].max()

    # Add the first line plot
    conflict_source = ColumnDataSource(df[df["TS"] == ts])
    conflict_line = p.line(
        "6-month-group",
        conflict_measure,
        source=conflict_source,
        legend_label=f"{conflict_measure.capitalize()}",
        line_width=2,
        color="red",
        alpha=0.7,
    )

    p.extra_y_ranges = {
        "y2": Range1d(
            start=df[df["TS"] == ts][ntl_measure].min(),
            end=df[df["TS"] == ts][ntl_measure].max(),
        )
    }
    # Add the second line plot
    ntl_source = ColumnDataSource(df[df["TS"] == ts])
    ntl_line = p.line(
        "6-month-group",
        ntl_measure,
        source=ntl_source,
        legend_label=ntl_measure,
        line_width=2,
        color="blue",
        alpha=0.7,
        y_range_name="y2",
    )

    # Customize legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

    p.add_layout(LinearAxis(y_range_name="y2", axis_label="Luminosity"), "right")

    # Add hover tool for conflict data
    hover_conflict = HoverTool(
        renderers=[conflict_line],  # Reference the renderer directly
        tooltips=[
            ("Date", "@6-month-group{%F}"),
            (f"{conflict_measure.capitalize()}", f"@{conflict_measure}"),
        ],
        formatters={
            "@6-month-group": "datetime",  # Format the date
        },
        mode="vline",
    )

    # Add hover tool for NTL data
    hover_ntl = HoverTool(
        renderers=[ntl_line],  # Reference the renderer directly
        tooltips=[
            ("Date", "@6-month-group{%F}"),
            (ntl_measure, f"@{ntl_measure}"),
        ],
        formatters={
            "@6-month-group": "datetime",  # Format the date
        },
        mode="vline",
    )

    p.add_tools(hover_conflict, hover_ntl)

    tab = TabPanel(child=p, title=ts)
    tabs.append(tab)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

### Comparing NTL and Conflict Increase from Baseline period of October 2020-March 2021

In [158]:
conflict_baseline_ts = (
    conflict_monthly[conflict_monthly["date"] < "2021-04-01"]
    .groupby(["TS"])[["conflict_index", "events", "fatalities"]]
    .mean()
    .reset_index()
)
conflict_baseline_ts.rename(
    columns={
        "conflict_index": "conflict_index_baseline",
        "events": "events_baseline",
        "fatalities": "fatalities_baseline",
    },
    inplace=True,
)
ntl_baseline_ts = (
    ntl_adm3_monthly[(ntl_adm3_monthly["date"] < "2021-04-01")]
    .groupby(["TS"])[["ntl_sum", "ntl_nogf_5km_sum", "ntl_gf_5km_sum"]]
    .mean()
    .reset_index()
)
ntl_baseline_ts.rename(
    columns={
        "ntl_sum": "ntl_sum_baseline",
        "ntl_nogf_5km_sum": "ntl_nogf_5km_sum_baseline",
        "ntl_gf_5km_sum": "ntl_gf_5km_sum_baseline",
    },
    inplace=True,
)

In [159]:
conflict_baseline_diff = conflict_monthly[
    ["TS", "date", "conflict_index", "events", "fatalities"]
]
conflict_baseline_diff = conflict_baseline_diff.merge(conflict_baseline_ts, on=["TS"])
conflict_baseline_diff["conflict_diff"] = (
    conflict_baseline_diff["conflict_index"]
    - conflict_baseline_diff["conflict_index_baseline"]
)

ntl_baseline_diff = ntl_adm3_monthly[
    ["TS", "date", "ntl_sum", "ntl_nogf_5km_sum", "ntl_gf_5km_sum"]
]
ntl_baseline_diff = ntl_baseline_diff.merge(ntl_baseline_ts, on=["TS"])
ntl_baseline_diff["ntl_nogf_5km_diff"] = (
    ntl_baseline_diff["ntl_nogf_5km_sum"]
    - ntl_baseline_diff["ntl_nogf_5km_sum_baseline"]
)

In [187]:
# Import necessary libraries
output_notebook()
from bokeh.models import (
    LinearAxis,
    Range1d,
    Tabs,
    TabPanel,
    HoverTool,
    ColumnDataSource,
)

df1 = ntl_baseline_diff
df1 = df1[df1["date"] > "2021-01-1"]
df2 = conflict_baseline_diff
conflict_measure = "conflict_diff"
ntl_measure = "ntl_nogf_5km_diff"


tabs = get_multi_tab_line_plot(
    df1, df2, very_severe_conflict, "TS", conflict_measure, ntl_measure
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...